In [110]:
from music21 import converter, instrument, note, chord
import glob
import pickle
import keras 
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint

from pdb import set_trace as bp

#TESTA SEN!!!!
#chord.chord.show()


In [112]:
class Bithoven:
    def __init__(self,seq_len=5, net_in=np.array([]), net_out=np.array([]), n_voc=0, w=0,n=[]):
        self.ns = seq_len     #number of sequences
        self.network_input = net_in
        self.network_output = net_out
        self.n_vocab = n_voc
        self.model = Sequential()
        self.weights = w
        self.notes = n
        self.song_len = 500
        
        self.prediction_output=[]
        self.pitchnames = 0
        self.int_From_note= 0
        
        print("end of construct")
        
    def get_data(self):
        
        for track in glob.glob("midi_songs/*.mid"):
            midi_track = converter.parse(track)
            to_be_parsed = 0
            different_instruments = instrument.partitionByInstrument(midi_track)
            
            if different_instruments:
                to_be_parsed = different_instruments.parts[0].recurse()
            else:
                to_be_parsed = midi_track.flat.notes
            
            for el in to_be_parsed:
                if isinstance(el, note.Note):
                    self.notes.append(str(el.pitch))
                elif isinstance(el,chord.Chord):
                    self.notes.append('.'.join(str(a_note) for a_note in el.normalOrder))
                    
#                     for a_note in el.normalOrder:
#                         self.notes.append('.'.join(str(a_note)))
        
        self.pitchnames = sorted(set(item for item in self.notes)) #GÖR OM TILL KLASSISK FOR-LOOP
        self.int_from_note = dict((note,number) for number,note in enumerate(self.pitchnames))
                        
        self.n_vocab = len(self.notes)
        print("notes")
        print(self.n_vocab)
    
    
    def prep_data(self, train_model=True):
        
        self.network_output = []
        self.network_input = np.zeros((1*(len(self.notes)-self.ns),self.ns,1))
#         bp()
        test_count = [];
        for i in range(0,len(self.notes)-self.ns,1):
             
            for j,characters in enumerate(self.notes[i:i+self.ns]):
                test_count.append(0)
#                 bp()
                self.network_input[i,j,0]= self.int_from_note[characters]
            self.network_output.append(self.int_from_note[ self.notes[i+self.ns] ])

        self.network_input /= self.n_vocab
        
        
        print("test_count")
        print(len(test_count))
#         bp()
#         if train_model:
# #             bp()
#             self.network_output = np_utils.to_categorical(self.network_output) #converts a class vector to a binary class matrix

            
            
        
        with open('data/notes', 'wb') as the_file:
#             bp()
            pickle.dump(self.notes,the_file)
        
        
    def create_rnn(self, train_model=True):
        self.model.add(LSTM(512,input_shape = (self.network_input.shape[1],self.network_input.shape[2]), return_sequences=True))
        self.model.add(Dropout(0.3))
        self.model.add(LSTM(512))
        self.model.add(Dense(256))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(self.n_vocab))
        self.model.add(Activation('softmax'))
        self.model.compile(loss='categorical_crossentropy',optimizer='rmsprop')
        
        if not train_model:
            self.model.load_weights('weights.hdf5')
        
    def train_rnn(self):
        
        the_file = "weights-improvement-{epoch:02d}--{loss:.4f}-bigger.hdf5"
        cpoint = ModelCheckpoint(the_file, monitor = 'loss', verbose=0,save_best_only = True, mode = 'min')
        cb_list = [cpoint] #callbacks
        output = np_utils.to_categorical(self.network_output)
        
        print("input")
        print(np.shape(self.network_input))
        print("output")        
        print(np.shape(output))
        print("cb")        
        print(len(cb_list))
#         a = input("ok?")        
        
        
        self.model.fit(self.network_input, output, epochs = 10, batch_size = 64, callbacks = cb_list)
        
        
        
        """ 
        
        NEDAN BÖRJAR GENERATE MUSIC  
        
        """
        
    def gen(self):
        with open('data/notes','rb') as the_file:
            notes = pickle.load(the_file)
        self.prep_data(train_model=False)
        unnormalized = self.network_input*self.n_vocab        
        self.create_rnn(train_model=False)
        self.create_music()
        self.gen_midi()
    
    def gen_midi(self):
        pause = 0
        notes_out = []
        
        """
        
        nn - new note
        nc - new chord
        
        """
        
        for pattern in self.prediction_output:
            
            if('.' in pattern) or pattern.isdigit():
                #if we enter here, we know that the pattern is a chord
                notes_in_chord = pattern.split('.')
                notes =[]
                
                for cur_note in notes_in_chord:
                    nn = note.Note(int(cur_note))
                    nn.storedInstrument = instrument.Piano()
                    notes.append(nn)
                    
                nc = chord.Chord(notes)
                nc.offset = pause
                notes_out.append(nc)
                
            else:
                #if we enter here, we know that the pattern is a note
                nn = note.Note(pattern)
                nn.offset=pause
                nn.storedInstrument = instrument.Piano()
                notes_out.append(nn)
                
            pause += 1.0/2
        
        stream_of_midi = stream.Stream(notes_out)
        stream_of_midi.write('midi', fp='generated_song.mid')
                

    
    def create_music(self):
        initiate = np.random.randint(0,len(self.network_input))
        
        pattern = network_input[initiate]
        predicted_out = []
        
        for j in range(self.song_len):
            
            predicted_in = np.reshape(pattern, (1,len(pattern),1 ) )
            predicted_in /= float(self.n_vocab)
            
            p = self.model.predict(predicted_in, verbose = 0)
            
            ind = np.argmax(p)
            
            res = self.int_from_note[ind]
            
            self.prediction_output.append(res)
            
            pattern.append(ind)
            pattern = pattern[1:len(pattern)]
            
        
    def main_train(self):
        self.get_data()
        self.prep_data()
        self.create_rnn()
        self.train_rnn()
        
    def main_generate(self):
        self.gen()
        
            
        

In [113]:
"""
RUN THIS CELL TO TRAIN THE NETWORK

"""

generator = Bithoven()
# try:
generator.main_train()    
# except KeyError as e:
#     print("exception")
#     pass


end of construct
notes
2997
test_count
14960
input
(2992, 5, 1)
output
(2992, 113)
cb
1


ValueError: Error when checking target: expected activation_25 to have shape (2997,) but got array with shape (113,)

In [21]:
"""
RUN THIS CELL TO GENERATE MUSIC

"""
generator.main_generate()



EOFError: Ran out of input

In [116]:
# len(generator.notes)
np.sum(np.sum(np_utils.to_categorical(generator.network_output)))

2992.0